In [35]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotZoo
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random
using Test
using Plots
include("quadratic_cost.jl")
include("walker.jl")
#include("sparseblocks.jl")
#include("utils.jl")
include("moi.jl") # Ipopt interface

  Activating environment at `~/Desktop/16745_OCRL/Project/Optimal-Control-Project/Project.toml`
Precompiling project...
  ✓ JSONSchema
  ✓ OpenSSL_jll
  ✓ ASL_jll
  ✓ METIS_jll
  ✓ OpenBLAS32_jll
  ✓ BenchmarkTools
  ✓ ChainRulesCore
  ✓ ArrayInterface
  ✓ FFMPEG_jll
  ✓ MUMPS_seq_jll
  ✓ StructArrays
  ✓ FFMPEG
  ✓ ChangesOfVariables
  ✓ WebIO
  ✓ Ipopt_jll
  ✓ FiniteDiff
  ✓ LogExpFunctions
  ✓ Interpolations
  ✓ JSExpr
  ✓ Blink
  ✓ SpecialFunctions
  ✓ GeometryBasics
  ✓ DiffRules
  ✓ MeshIO
  ✓ ForwardDiff
  ✓ MeshCat
  ✓ SparseDiffTools
  ✓ RobotDynamics
  ✓ RobotZoo
  ✓ MathOptInterface
  ✓ Ipopt
  ✓ OSQP
  ✓ TrajectoryOptimization
  ✓ TrajOptPlots
  ✓ Altro
  35 dependencies successfully precompiled in 66 seconds (123 already precompiled)
  20 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions
┌ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1342
┌ Info: Precompiling TrajOptPlots [7

LoadError: ArgumentError: Package Plots not found in current path:
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.


In [36]:
model = UnitreeA1()
n,m = state_dim(model), control_dim(model)
mvis = initialize_visualizer(model)
render(mvis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8702
└ @ MeshCat /home/astutt/.julia/packages/MeshCat/Ax8pH/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x77, 0x6f, 0x72, 0x6c, 0x64, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("trunk" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x62, 0x61, 0x73, 0x65, 0x2f, 0x74, 0x72, 0x75, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_floating_base" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x61, 0x74, 0x69, 0x6e, 0x67, 0x5f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "RL_hip" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x75, 0x6e, 0x6b, 0x2f, 0x52, 0x4c, 0x5f, 0x68, 0x69, 0x70], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("RL_thigh" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x70, 0x2f, 0x52, 0x4c, 0x5f, 0x74, 0x68, 0x69, 0x67, 0x68], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_RL_thigh_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x69, 0x67, 0x68, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_17" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x37], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x37], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "RL_calf" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x68, 0x2f, 0x52, 0x4c, 0x5f, 0x63, 0x61, 0x6c, 0x66], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("RL_foot" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6c, 0x66, 0x2f, 0x52, 0x4c, 0x5f, 0x66, 0x6f, 0x6f, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_RL_foot_fixed" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x6f, 0x6f, 0x74, 0x5f, 0x66, 0x69, 0x78, 0x65, 0x64], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_19" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x39], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x39], Dict{String, Vector{UInt8}}(), n

## Cost Fxn

Stealing from HW4, quadractic_cost.jl

$$ \begin{bmatrix}
c_\text{init} \\
c_\text{goal} \\
c_\text{dynamics} \\
c_\text{stance} \\
c_\text{length} \\
\end{bmatrix}$$
which are of length $n$, $n$, $(N-1)n$, $N$, and $2N$, respectively. The dynamics, stance, and length constraints should be ordered by time step.

In [30]:
"""
    HybridNLP{n,m,L,Q}

Represents a (N)on(L)inear (P)rogram of a trajectory optimization problem,
with a dynamics model of type `L`, a quadratic cost function, horizon `T`, 
and initial and final state `x0`, `xf`.

The kth state and control can be extracted from the concatenated state vector `Z` using
`Z[nlp.xinds[k]]`, and `Z[nlp.uinds[k]]`.

# Constructor
    HybridNLP(model, obj, tf, N, M, x0, xf, [integration])

# Basic Methods
    Base.size(nlp)    # returns (n,m,T)
    num_ineq(nlp)     # number of inequality constraints
    num_eq(nlp)       # number of equality constraints
    num_primals(nlp)  # number of primal variables
    num_duals(nlp)    # total number of dual variables
    packZ(nlp, X, U)  # Stacks state `X` and controls `U` into one vector `Z`

# Evaluating the NLP
The NLP supports the following API for evaluating various pieces of the NLP:

    eval_f(nlp, Z)         # evaluate the objective
    grad_f!(nlp, grad, Z)  # gradient of the objective
    eval_c!(nlp, c, Z)     # evaluate the constraints
    jac_c!(nlp, c, Z)      # constraint Jacobian
"""
struct HybridNLP{n,m,L,Q} <: MOI.AbstractNLPEvaluator
    model::L                                 # dynamics model
    obj::Vector{QuadraticCost{n,m,Float64}}  # objective function
    N::Int                                   # number of knot points
    M::Int                                   # number of steps in each mode
    Nmodes::Int                              # number of modes
    tf::Float64                              # total time (sec)
    x0::MVector{n,Float64}                   # initial condition
    xf::MVector{n,Float64}                   # final condition
    times::Vector{Float64}                   # vector of times
    modes::Vector{Int}                       # mode ID
    xinds::Vector{SVector{n,Int}}            # Z[xinds[k]] gives states for time step k
    uinds::Vector{SVector{m,Int}}            # Z[uinds[k]] gives controls for time step k
    cinds::Vector{UnitRange{Int}}            # indices for each of the constraints
    lb::Vector{Float64}                      # lower bounds on the constraints
    ub::Vector{Float64}                      # upper bounds on the constraints
    zL::Vector{Float64}                      # lower bounds on the primal variables
    zU::Vector{Float64}                      # upper bounds on the primal variables
    rows::Vector{Int}                        # rows for Jacobian sparsity
    cols::Vector{Int}                        # columns for Jacobian sparsity
    use_sparse_jacobian::Bool
    blocks::BlockViews
    function HybridNLP(model, obj::Vector{<:QuadraticCost{n,m}},
            tf::Real, N::Integer, M::Integer, x0::AbstractVector, xf::AbstractVector, 
            integration::Type{<:QuadratureRule}=RK4; use_sparse_jacobian::Bool=false
        ) where {n,m}
        # Create indices
        xinds = [SVector{n}((k-1)*(n+m) .+ (1:n)) for k = 1:N]
        uinds = [SVector{m}((k-1)*(n+m) .+ (n+1:n+m)) for k = 1:N-1]
        times = collect(range(0, tf, length=N))
        
        # Specify the mode sequence
        modes = map(1:N) do k
            isodd((k-1) ÷ M + 1) ? 1 : 2
        end
        Nmodes = Int(ceil(N/M))
        
        # TODO: specify the constraint indices
        c_init_inds = 1:n          # initial constraint
        c_term_inds = (c_init_inds[end]+1):(c_init_inds[end]+n)          # terminal constraint
        c_dyn_inds = (c_term_inds[end]+1):(c_term_inds[end]+n*(N-1))           # dynamics constraints
        c_stance_inds = (c_dyn_inds[end]+1):(c_dyn_inds[end]+N)        # stance constraint (1 per time step)
        c_length_inds = (c_stance_inds[end]+1):(c_stance_inds[end]+(2*N))        # length bounds     (2 per time step)
        
        m_nlp = c_length_inds.stop # total number of constraints
        
        # TODO: specify the bounds on the constraints
        #lb = fill(+Inf,m_nlp)                                                # lower bounds on the constraints
        #ub = fill(-Inf,m_nlp)                                                # upper bounds on the constraints
        
        lb = zeros(m_nlp)
        lb[c_length_inds] .= 0.5 # min length
        
        ub = zeros(m_nlp)
        ub[c_length_inds] .= 1.5 # max length
        

        # Other initialization
        cinds = [c_init_inds, c_term_inds, c_dyn_inds, c_stance_inds, c_length_inds]
        n_nlp = n*N + (N-1)*m
        zL = fill(-Inf, n_nlp)
        zU = fill(+Inf, n_nlp)
        rows = Int[]
        cols = Int[]
        blocks = BlockViews(m_nlp, n_nlp)
        
        new{n,m,typeof(model), integration}(
            model, obj,
            N, M, Nmodes, tf, x0, xf, times, modes,
            xinds, uinds, cinds, lb, ub, zL, zU, rows, cols, use_sparse_jacobian, blocks
        )
    end
end
Base.size(nlp::HybridNLP{n,m}) where {n,m} = (n,m,nlp.N)
num_primals(nlp::HybridNLP{n,m}) where {n,m} = n*nlp.N + m*(nlp.N-1)
num_duals(nlp::HybridNLP) = nlp.cinds[end][end]

"""
    packZ(nlp, X, U)

Take a vector state vectors `X` and controls `U` and stack them into a single vector Z.
"""
function packZ(nlp, X, U)
    Z = zeros(num_primals(nlp))
    for k = 1:nlp.N-1
        Z[nlp.xinds[k]] = X[k]
        Z[nlp.uinds[k]] = U[k]
    end
    Z[nlp.xinds[end]] = X[end]
    return Z
end

"""
    unpackZ(nlp, Z)

Take a vector of all the states and controls and return a vector of state vectors `X` and
controls `U`.
"""
function unpackZ(nlp, Z)
    X = [Z[xi] for xi in nlp.xinds]
    U = [Z[ui] for ui in nlp.uinds]
    return X, U
end

function TrajOptPlots.visualize!(vis, nlp::HybridNLP, Z)
    TrajOptPlots.visualize!(vis, nlp.model, nlp.tf, unpackZ(nlp, Z)[1])
end

# includes the interface to Ipopt
include("moi.jl")

LoadError: UndefVarError: MOI not defined

In [3]:
# steal hybrid NLP struct from homework, or define our own??

"""
    eval_f(nlp, Z)

Evaluate the objective, returning a scalar.
"""
function eval_f(nlp::HybridNLP, Z)
    J = 0.0
    xi,ui = nlp.xinds, nlp.uinds
    for k = 1:nlp.N-1
        x,u = Z[xi[k]], Z[ui[k]]
        J += stagecost(nlp.obj[k], x, u)
    end
    J += termcost(nlp.obj[end], Z[xi[end]])
    return J
end

"""
    grad_f!(nlp, grad, Z)

Evaluate the gradient of the objective at `Z`, storing the result in `grad`.
"""
function grad_f!(nlp::HybridNLP{n,m}, grad, Z) where {n,m}
    xi,ui = nlp.xinds, nlp.uinds
    obj = nlp.obj
    for k = 1:nlp.N-1
        x,u = Z[xi[k]], Z[ui[k]]
        grad[xi[k]] = obj[k].Q*x + obj[k].q
        grad[ui[k]] = obj[k].R*u + obj[k].r
    end
    grad[xi[end]] = obj[end].Q*Z[xi[end]] + obj[end].q
    return nothing
end

LoadError: UndefVarError: HybridNLP not defined

## Reference Trajectory

In [6]:
function reference_trajectory(model::UnitreeA1, times;
        xinit = 0.0,
        xterm = 10.0,
        height = 1.0, # MODIFY LATER based on leg lengths
    )
    
    # Some useful variables
    n,m = size(model)
    tf = times[end]
    N = length(times)
    Δx = xterm - xinit
    
    # NEED TO SET THESE IN OUT MODEL SCTRUCT
    mb,g = model.mb, model.g
    body_width = model.body_width
    body_length = model.body_length
    
    # initialization
    xref = zeros(n,N)
    uref = zeros(m,N)
    
    # linearly interpolate x-pos
    # constant y-pos
    # velocities by taking difference
    
    xs = range(xinit,xterm,length=N)
    
    dt = Δx / tf
    
    # do this smarter in the future
    # to make a less boring trajectory
    
    for k = 1:N-1
          
        xref[1,k] = xs[k] # body x
        xref[2,k] = 0 # body y
        xref[3,k] = height # body z
        
        # no body rotation in reference
        
        # foot index: (1,2,3,4) = (FL,BL,FR,BR)
        
        # foot x pos
        xref[8,k] = xs[k] + (body_length/2)
        xref[11,k] = xs[k] - (body_length/2)
        xref[14,k] = xs[k] + (body_length/2)
        xref[17,k] = xs[k] - (body_length/2)
        
        # foot y pos
        xref[9,k] = -body_width / 2
        xref[12,k] = -body_width / 2
        xref[15,k] = body_width / 2
        xref[18,k] = body_width / 2
        
        # foot z pos
        # keep at 0

        xref[20,k] = (xs[k+1] - xs[k])/dt # body x vel
        xref[26,k] = (xs[k+1] - xs[k])/dt # foot1 x vel
        xref[29,k] = (xs[k+1] - xs[k])/dt # foot2 x vel
        xref[32,k] = (xs[k+1] - xs[k])/dt # foot3 x vel
        xref[35,k] = (xs[k+1] - xs[k])/dt # foot4 x vel
    end
    
    # end state
    # set terminal positions, all velocities should 0 out
    
    # foot x pos
    xref[8,N] = xterm + (body_length/2)
    xref[11,N] = xterm - (body_length/2)
    xref[14,N] = xterm + (body_length/2)
    xref[17,N] = xterm - (body_length/2)

    # foot y pos
    xref[9,N] = -body_width / 2
    xref[12,N] = -body_width / 2
    xref[15,N] = body_width / 2
    xref[18,N] = body_width / 2
    
    
    # reference trajectory
    uref .= kron(ones(N)', [0.5*mb*g; 0.5*mb*g; 0.5*mb*g; 0.5*mb*g]) # 1/2mg for each foot
    
    # Convert to a trajectory
    Xref = [SVector{n}(x) for x in eachcol(xref)]
    Uref = [SVector{m}(u) for u in eachcol(uref)]
    return Xref, Uref
end

reference_trajectory (generic function with 1 method)

In [7]:
# Discretization
tf = 4.4
dt = 0.1
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N);

# Reference Trajectory
Xref,Uref = reference_trajectory(model, times)

# ALL THE MODEL PARAMS ARE WRONG
# so nothing will actually work rn

LoadError: BoundsError: attempt to access 28×45 Matrix{Float64} at index [29, 1]

## Contact Sequence / Gait

In [29]:
# Discretization
tf = 4.4
dt = 0.1
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N);

function contact_sequence_trot(times)
    # 1 = stance, 0 = swing
    # 4xN matrix encoding contact schedule
    
    # initialize
    contact_schedule = zeros(4,length(times))

    step_length = 5 # NOTE: check later, is this physically feasible w/ our dt

    #hardcoded a trot
    for k = 1:N
        if k % (2*step_length) < step_length
            contact_schedule[:,k] .= [1;0;0;1]
        else
            contact_schedule[:,k] .= [0;1;1;0]
        end
    end
    return contact_schedule
end

contact_sched = contact_sequence_trot(times)

4×45 Matrix{Float64}:
 1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  …  0.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0     1.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0     1.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0     0.0  1.0  1.0  1.0  1.0  1.0  0.0